In [1]:
import numpy as np

In [2]:
import os
import torch
from torch.utils.data import DataLoader, random_split
from lightning.pytorch import Trainer

/home/demancum/miniconda3/envs/cocola/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
os.environ['CUDA_VISIBLE_DEVICES'] = '0'  

In [4]:
from contrastive_model.contrastive_model import CoCola
from data import (
musdb_contrastive_preprocessed,
moisesdb_contrastive_preprocessed,
slakh2100_contrastive_preprocessed,
coco_chorales_contrastive_preprocessed
)


In [5]:
#CHECKPOINT = '/speech/dbwork/mul/spielwiese4/students/demancum/cocola/k5llcdm8/checkpoints/epoch=21-step=21406.ckpt' #bumbling-breeze-34 bs=512 chunk_dur=2s
#CHECKPOINT = '/speech/dbwork/mul/spielwiese4/students/demancum/cocola/r71kxj7r/checkpoints/epoch=36-step=29119.ckpt' #good-wave-24 bs=256 chunk_dur=5s
#CHECKPOINT = '/speech/dbwork/mul/spielwiese4/students/demancum/cocola/shov44gs/checkpoints/epoch=24-step=19900.ckpt' #golden-dew-25 bs=128 chunk_dur=10s

#CHECKPOINT = '/speech/dbwork/mul/spielwiese4/students/demancum/cocola/zr304efn/checkpoints/epoch=23-step=17448.ckpt' #flowing-sound-26 bs=96 chunk_dur=15s

#CHECKPOINT = '/speech/dbwork/mul/spielwiese4/students/demancum/cocola/8jtiowyz/checkpoints/epoch=46-step=38963.ckpt' #jumping-breeze-27 bs=64 chunk_dur=20s

CHECKPOINT = '/speech/dbwork/mul/spielwiese3/demancum/cocola/e3xubuzt/checkpoints/epoch=38-step=245622.ckpt' #frosty-silence-39 musdb 32bs test


model = CoCola.load_from_checkpoint(CHECKPOINT)
trainer = Trainer()


/home/demancum/miniconda3/envs/cocola/lib/python3.11/site-packages/lightning/fabric/plugins/environments/slurm.py:204: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /home/demancum/miniconda3/envs/cocola/lib/python3.11 ...
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


## MOISESDB

In [18]:
dataset = moisesdb_contrastive_preprocessed.get_dataset(
    chunk_duration=5,
    positive_noise=0,
    generate_submixtures=True,
    transform=None
    )

_, _, test_dataset = random_split(
    dataset=dataset,
    lengths=[0.8, 0.1, 0.1],
    generator=torch.Generator().manual_seed(42)
    )

test_dataloader = DataLoader(
    test_dataset,
    batch_size=2,
    shuffle=False,
    drop_last=True,
    num_workers=os.cpu_count(),
    persistent_workers=True
    )


INFO:Found original dataset at /disk1/demancum/moisesdb/moisesdb_v0.1.
INFO:Found preprocessed dataset at /disk1/demancum/moisesdb/preprocessed5.


In [19]:
trainer.test(model=model, dataloaders=test_dataloader)


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing DataLoader 0: 100%|██████████| 1043/1043 [00:14<00:00, 72.16it/s]


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│       test_accuracy       │    0.9419942473633748     │
│         test_loss         │    0.17246873676776886    │
└───────────────────────────┴───────────────────────────┘

[{'test_loss': 0.17246873676776886, 'test_accuracy': 0.9419942473633748}]

## Reults MOISESDB

bumbling-breeze-34 bs=512 chunl_dur=2s test on moisesdb bs=512 [{'test_loss': 30.519054412841797, 'test_accuracy': 0.396484375}]

bumbling-breeze-34 bs=512 chunl_dur=2s test on moisesdb bs=64 [{'test_loss': 29.30805206298828, 'test_accuracy': 0.5985725308641975}]

bumbling-breeze-34 bs=512 chunl_dur=2s test on moisesdb bs=2 [{'test_loss': 25.239595413208008, 'test_accuracy': 0.8775077160493827}]

---

good-wave-24 bs=256 chunl_dur=5s, test on moisesdb bs=256 [{'test_loss': 2.751087188720703, 'test_accuracy': 0.41650390625}] 

good-wave-24 bs=256 chunl_dur=5s, test on moisesdb bs=64 [{'test_loss': 1.7366595268249512, 'test_accuracy': 0.5849609375}]

good-wave-24 bs=256 chunl_dur=5s, test on moisesdb bs=2 [{'test_loss': 0.1633053719997406, 'test_accuracy': 0.9415148609779481}]

---

golden-dew-25 bs=128 chunk_dur=10s test on moisesdb bs=128 [{'test_loss': 14.337109565734863, 'test_accuracy': 0.35379464285714285}]

golden-dew-25 bs=128 chunk_dur=10s test on moisesdb bs=64 [{'test_loss': 13.74421501159668, 'test_accuracy': 0.4654017857142857}]

golden-dew-25 bs=128 chunk_dur=10s test on moisesdb bs=2 [{'test_loss': 10.538426399230957, 'test_accuracy': 0.8828451882845187}]

---
flowing-sound-26 bs=96 chunl_dur=15s test on moisesdb bs=96 [{'test_loss': 3.2646923065185547, 'test_accuracy': 0.24702380952380953}]

flowing-sound-26 bs=96 chunl_dur=15s test on moisesdb bs=64 [{'test_loss': 2.8988101482391357, 'test_accuracy': 0.28267045454545453}]

flowing-sound-26 bs=96 chunl_dur=15s test on moisesdb bs=2 [{'test_loss': 0.29432547092437744, 'test_accuracy': 0.8728813559322034}]

---

jumping-breeze-27 bs=64 chunl_dur=20s test on moisesdb bs=64 [{'test_loss': 3.286912441253662, 'test_accuracy': 0.38671875}]

jumping-breeze-27 bs=64 chunl_dur=20s test on moisesdb bs=2 [{'test_loss': 0.5389121770858765, 'test_accuracy': 0.8936567164179104}]

---

frosty-silence-39 bs=32 chunk_dur=5s test on moisesdb bs=64 [{'test_loss': 1.6719969511032104, 'test_accuracy': 0.60546875}]

frosty-silence-39 bs=32 chunk_dur=5s test on moisesdb bs=2  [{'test_loss': 0.17246873676776886, 'test_accuracy': 0.9419942473633748}]



## slakh2100

In [13]:
test_dataset = slakh2100_contrastive_preprocessed.get_dataset(
    split="test",
    chunk_duration=5,
    positive_noise=0,
    generate_submixtures=True,
    transform=None
    )

test_dataloader = DataLoader(
    test_dataset,
    batch_size=2,
    shuffle=False,
    drop_last=True,
    num_workers=os.cpu_count(),
    persistent_workers=True
    )

trainer.test(model=model, dataloaders=test_dataloader)

INFO:Found original dataset split test at /disk1/demancum/slakh2100_contrastive/original/slakh2100_redux_16k/test.
INFO:Found preprocessed dataset split test at /disk1/demancum/slakh2100_contrastive/preprocessed5/test.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing DataLoader 0: 100%|██████████| 7946/7946 [01:52<00:00, 70.93it/s]


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│       test_accuracy       │    0.9336143971809716     │
│         test_loss         │    0.15587395429611206    │
└───────────────────────────┴───────────────────────────┘

[{'test_loss': 0.15587395429611206, 'test_accuracy': 0.9336143971809716}]

## Results slakh2100

bumbling-breeze-34 bs=512 chunk_dur=2s test on slakh2100 bs=64 [{'test_loss': 2.6327898502349854, 'test_accuracy': 0.5836530694668821}]

bumbling-breeze-34 bs=512 chunk_dur=2s test on slakh2100 bs=2 [{'test_loss': 1.0304263830184937, 'test_accuracy': 0.9223624432104998}]

--- 

good-wave-24 bs=256 chunk_dur=5s test on slakh2100 bs=64 [{'test_loss': 2.032883405685425, 'test_accuracy': 0.5696194556451613}]

good-wave-24 bs=256 chunk_dur=5s test on slakh2100 bs=2 [{'test_loss': 0.24336093664169312, 'test_accuracy': 0.9187012333249435}]

---

golden-dew-25 bs=128 chunk_dur=10s test on slakh2100 bs=64 [{'test_loss': 1.9303451776504517, 'test_accuracy': 0.5871975806451613}]

golden-dew-25 bs=128 chunk_dur=10s test on slakh2100 bs=2 [{'test_loss': 0.18359966576099396, 'test_accuracy': 0.9335086401202103}]

---

flowing-sound-26 bs=96 chunk_dur=15s test on slakh2100 bs=64 [{'test_loss': 2.262129068374634, 'test_accuracy': 0.4759036144578313}]

flowing-sound-26 bs=96 chunk_dur=15s test on slakh2100 bs=2 [{'test_loss': 0.1931096464395523, 'test_accuracy': 0.9245883233532934}]

---
frosty-silence-39 bs=32 chunk_dur=5s test on slakh2100 bs=64  [{'test_loss': 1.6478705406188965, 'test_accuracy': 0.612273185483871}]

frosty-silence-39 bs=32 chunk_dur=5s test on slakh2100 bs=2  [{'test_loss': 0.15587395429611206, 'test_accuracy': 0.9336143971809716}]


## COCOCHORALES

In [10]:
test_dataset = coco_chorales_contrastive_preprocessed.get_dataset(
    split="test",
    ensemble="*",
    chunk_duration=5,
    positive_noise=0,
    generate_submixtures=True,
    transform=None
    )
    
test_dataloader = DataLoader(
    test_dataset,
    batch_size=64,
    shuffle=False,
    drop_last=True,
    num_workers=os.cpu_count(),
    persistent_workers=True
    )


INFO:Found original dataset split test at /disk1/demancum/coco_chorales_contrastive/original/test.
INFO:Found preprocessed dataset split test at /disk1/demancum/coco_chorales_contrastive/preprocessed5/test.


In [11]:
trainer.test(model=model, dataloaders=test_dataloader)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing DataLoader 0: 100%|██████████| 291/291 [00:16<00:00, 17.50it/s]


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│       test_accuracy       │    0.9119415807560137     │
│         test_loss         │    0.2683464288711548     │
└───────────────────────────┴───────────────────────────┘

[{'test_loss': 0.2683464288711548, 'test_accuracy': 0.9119415807560137}]

## Results COCOCHORALES

bumbling-breeze-34 bs=512 chunk_dur=2s test on COCOCHORALES bs=64 [{'test_loss': 3.28816819190979, 'test_accuracy': 0.9072003182461104}]

bumbling-breeze-34 bs=512 chunk_dur=2s test on COCOCHORALES bs=2 [{'test_loss': 2.5330116748809814, 'test_accuracy': 0.9808440123729563}]

---

good-wave-24 bs=256 chunk_dur=5s test on COCOCHORALES bs=64 [{'test_loss': 0.24012035131454468, 'test_accuracy': 0.9311640893470791}]

good-wave-24 bs=256 chunk_dur=5s test on COCOCHORALES bs=2 [{'test_loss': 0.011269009672105312, 'test_accuracy': 0.997055043906618}]

---

golden-dew-25 bs=128 chunk_dur=10s test on COCOCHORALES bs=64 [{'test_loss': 1.371890902519226, 'test_accuracy': 0.9197303921568627}]

golden-dew-25 bs=128 chunk_dur=10s test on COCOCHORALES bs=2 [{'test_loss': 0.5721641182899475, 'test_accuracy': 0.9725274725274725}]

---
flowing-sound-26 bs=96 chunk_dur=15s test on COCOCHORALES bs=64 [{'test_loss': 0.1298881322145462, 'test_accuracy': 0.9505208333333334}]

flowing-sound-26 bs=96 chunk_dur=15s test on COCOCHORALES bs=2 [{'test_loss': 0.0018012290820479393, 'test_accuracy': 0.9992791234140715}]

---
frosty-silence-39 bs=32 chunk_dur=5s test on COCOCHORALES bs=64 [{'test_loss': 0.2683464288711548, 'test_accuracy': 0.9119415807560137}]

frosty-silence-39 bs=32 chunk_dur=5s test on COCOCHORALES bs=2 [{'test_loss': 0.010394576005637646, 'test_accuracy': 0.9968408652816448}]



## MusDB


In [7]:
# Using train split since it is not used from training
test_dataset = musdb_contrastive_preprocessed.get_dataset(
    split="train",
    chunk_duration=5,
    positive_noise=0,
    generate_submixtures=True,
    transform=None
    )

test_dataloader = DataLoader(
    test_dataset,
    batch_size=64,
    shuffle=False,
    drop_last=True,
    num_workers=os.cpu_count(),
    persistent_workers=True
    )

trainer.test(model=model, dataloaders=test_dataloader)



INFO:Found original dataset split train at /disk1/demancum/musdb_contrastive/original/train.
INFO:Found preprocessed dataset split train at /disk1/demancum/musdb_contrastive/preprocessed5/train.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing DataLoader 0: 100%|██████████| 144/144 [00:08<00:00, 16.25it/s]


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│       test_accuracy       │    0.5073784722222222     │
│         test_loss         │     2.596252202987671     │
└───────────────────────────┴───────────────────────────┘

[{'test_loss': 2.596252202987671, 'test_accuracy': 0.5073784722222222}]

## Results MusDB

bumbling-breeze-34 bs=512 chunk_dur=2s test on MusDB bs=64 [{'test_loss': 17.289331436157227, 'test_accuracy': 0.4112255586592179}]

bumbling-breeze-34 bs=512 chunk_dur=2s test on MusDB bs=2 [{'test_loss': 13.33316707611084, 'test_accuracy': 0.8486590038314177}]

---

good-wave-24 bs=256 chunk_dur=5s test on MusDB bs=64 [{'test_loss': 4.149162769317627, 'test_accuracy': 0.3428819444444444}]

good-wave-24 bs=256 chunk_dur=5s test on MusDB bs=2 [{'test_loss': 0.6049458980560303, 'test_accuracy': 0.8654470664646027}]

---

golden-dew-25 bs=128 chunk_dur=10s test on MusDB bs=64 [{'test_loss': 12.250128746032715, 'test_accuracy': 0.296875}]


golden-dew-25 bs=128 chunk_dur=10s test on MusDB bs=2 [{'test_loss': 6.9717888832092285, 'test_accuracy': 0.8437097466723916}]

---

flowing-sound-26 bs=96 chunk_dur=15s test on MusDB bs=64 [{'test_loss': 4.6844329833984375, 'test_accuracy': 0.2688802083333333}]

flowing-sound-26 bs=96 chunk_dur=15s test on MusDB bs=2 [{'test_loss': 0.8233780264854431, 'test_accuracy': 0.7988505747126438}]

---

frosty-silence-39 bs=32 chunk_dur=5s test on MusDB bs=64 [{'test_loss': 2.596252202987671, 'test_accuracy': 0.5073784722222222}]

frosty-silence-39 bs=32 chunk_dur=5s test on MusDB bs=2 [{'test_loss': 0.30304449796676636, 'test_accuracy': 0.9180558562459407}]